# 7.6 残差网络（ResNet）
- **目录**
  - 7.6.1 函数类
  - 7.6.2 残差块
  - 7.6.3 ResNet模型
  - 7.6.4 训练模型

随着我们设计越来越深的网络，深刻理解“**新添加的层如何提升神经网络的性能**”变得至关重要。更重要的是设计网络的能力，在这种网络中，添加层会使网络更具表现力，
为了取得质的突破，我们需要一些数学基础知识。

## 7.6.1 函数类

所谓函数类是一组**具有相似结构或性质的函数的集合**。

首先，假设有一类特定的神经网络架构$\mathcal{F}$，它包括学习速率和其他超参数设置。
对于所有$f \in \mathcal{F}$，存在一些参数集（例如权重和偏置），这些参数可以通过在合适的数据集上进行训练而获得。
现在假设$f^*$是我们真正想要找到的函数，如果是$f^* \in \mathcal{F}$，那我们可以轻而易举的训练得到它，但通常我们不会那么幸运。
相反，我们将尝试找到一个函数$f^*_\mathcal{F}$，这是我们在$\mathcal{F}$中的最佳选择。
例如，给定一个具有$\mathbf{X}$特性和$\mathbf{y}$标签的数据集，我们可以尝试通过解决以下优化问题来找到它：

$$f^*_\mathcal{F} := \mathop{\mathrm{argmin}}_f L(\mathbf{X}, \mathbf{y}, f) \text{ subject to } f \in \mathcal{F}. \tag{7.6.1}$$

那么，怎样得到更近似真正$f^*$的函数呢？
唯一合理的可能性是，我们需要设计一个<b>更强大的架构$\mathcal{F}'$ </b>。
换句话说，我们预计$f^*_{\mathcal{F}'}$比$f^*_{\mathcal{F}}$**更近似** 。
然而，如果$\mathcal{F} \not\subseteq \mathcal{F}'$，则无法保证新的体系“更近似”。
事实上，$f^*_{\mathcal{F}'}$可能更糟：
如图7.6.1所示，对于非嵌套函数（non-nested function）类，较复杂的函数类并不总是向“真”函数$f^*$靠拢（复杂度由$\mathcal{F}_1$向$\mathcal{F}_6$递增）。
在图7.6.1的左边，虽然$\mathcal{F}_3$比$\mathcal{F}_1$更接近$f^*$，但$\mathcal{F}_6$却离的更远了。
相反对于图7.6.1右侧的嵌套函数（nested function）类$\mathcal{F}_1 \subseteq \ldots \subseteq \mathcal{F}_6$，我们可以避免上述问题。


<center>
    <img src="../img/functionclasses.svg" alt="一个单隐藏层的多层感知机，具有5个隐藏单元">
</center>
<center>
    图7.6.1 对于非嵌套函数类，较复杂（由较大区域表示）的函数类不能保证更接近“真”函数（$f^*$）。<br>这种现象在嵌套函数类中不会发生
</center>
<br>

因此，只有当较复杂的函数类包含较小的函数类时，我们才能确保提高它们的性能。
对于深度神经网络，如果我们能将新添加的层训练成**恒等映射（identity function）** $f(\mathbf{x}) = \mathbf{x}$，新模型和原模型将同样有效。
同时，由于新模型可能得出更优的解来拟合训练数据集，**因此添加层似乎更容易降低训练误差**。

针对这一问题，何恺明等人提出了**残差网络（ResNet）**(He et al., 2016a
He, K., Zhang, X., Ren, S., & Sun, J. (2016). Deep residual learning for image recognition. Proceedings of the IEEE conference on computer vision and pattern recognition (pp. 770–778).)。
它在2015年的ImageNet图像识别挑战赛夺魁，并深刻影响了后来的深度神经网络的设计。
残差网络的核心思想是：**每个附加层都应该更容易地包含原始函数作为其元素之一**。
于是，**残差块（residual blocks）** 便诞生了，这个设计对如何建立深层神经网络产生了深远的影响。
凭借它，ResNet赢得了2015年ImageNet大规模视觉识别挑战赛。

- **要点：**
  -  **函数类:** 在神经网络架构中，存在一类特定的神经网络结构$\mathcal{F}$，包括学习率和其他超参数设置。对于所有$f \in \mathcal{F}$，通过在合适的数据集上进行训练，可以得到一些参数集（例如权重和偏置）。我们希望找到一个函数$f^*_\mathcal{F}$，这是我们在$\mathcal{F}$中的最佳选择。
  - **非嵌套与嵌套函数类:** 对于非嵌套函数类，即使函数类变得更复杂，也不能保证找到的函数更接近真实的函数。但是，对于嵌套函数类，较复杂的函数类能够包含较简单的函数类，因此可以提高性能。
  - **深度神经网络的挑战：** 尽管通常认为网络越深，模型的性能越好，但实际情况并非如此。当网络过深时，训练难度会增加，准确率可能下降，这被称为“退化”问题。
  - **残差网络（ResNet）的解决方案：** 为了解决这个问题，何恺明等人提出了残差网络（ResNet）和残差块的概念。ResNet的核心思想是每个附加层都应该更容易地包含原始函数作为其元素之一。这样，新添加的层可以被训练成恒等映射，新模型和原模型将同样有效，同时新模型可能找到更优的解来拟合训练数据，从而更容易降低训练误差。
  - **ResNet的影响：** 残差网络的这种创新设计赢得了2015年ImageNet图像识别挑战赛，并对后续深度神经网络的设计产生了深远影响。

----------------
- **说明：嵌套函数类与ResNet**
  - 首先，我们需要理解什么是函数类。在机器学习或深度学习中，函数类可以被视为**一组特定的模型或算法**，它们可能拥有不同的参数设置、结构或者超参数。例如，所有3层神经网络组成的集合可视为一个函数类，所有5层神经网络组成的集合可视为另一个函数类。
  - **嵌套函数类**： 如果一个函数类中的所有函数都能在另一个函数类中找到，那么这两个函数类就有嵌套关系，我们称其中的一个函数类嵌套在另一个函数类中。比如，我们可以将所有三层神经网络的集合看作是所有五层神经网络集合的一个子集，因为我们可以通过将五层神经网络中的一些层设置为**恒等映射**，将五层神经网络转换为三层神经网络。在这种情况下，如果我们增加模型的复杂性（如增加神经网络的深度），我们至少可以保持与原来相同的性能，因为原来的模型已经包含在新的、更复杂的函数类中。此外，更复杂的模型还可能找到更好的函数来拟合数据，从而提高性能。
  - 这就是残差网络背后的核心思想：通过引入**跳跃连接和恒等映射**，ResNet构建了一系列**嵌套的函数类**，使得更深的网络可以至少达到与浅层网络相同的性能，并有可能通过额外的层找到更好的解，从而提高模型的性能。
  - **跳跃连接（Skip Connections）和恒等映射（Identity Mapping）**：在神经网络中，跳跃连接的概念是让某一层的输入直接传给更高层的输出。这种机制让信息可以直接从较低的层传到较高的层，而不需要经过每一个中间层，有效地解决了深度神经网络难以训练的问题。
  - 残差网络（ResNet）中的恒等映射是通过这种跳跃连接实现的。具体来说，假设我们有一个浅层网络，它的输入为$x$，输出为$H(x)$。当我们向这个网络添加新的层时，新的层的输入为$H(x)$，输出为$F(H(x)) + H(x)$，其中$F(H(x))$是新添加的层对输入$H(x)$进行变换后得到的结果。
  - 这里的$F(H(x)) + H(x)$就是所谓的恒等映射。如果新添加的层能够学习到一个**零映射**，即$F(H(x)) = 0$，那么整个网络的输出就会退化为$H(x)$，也就是说，新的网络至少可以达到与原来浅层网络相同的性能。这就保证了深度网络的性能不会因为增加新的层而降低。
  - **构建嵌套函数类**：基于上述的设计，ResNet构建了一系列嵌套的函数类。具体来说，假设我们有一个三层的神经网络，它的函数类记为$\mathcal{F}_3$。当我们向这个网络添加两个新的层时，我们得到一个五层的神经网络，其函数类记为$\mathcal{F}_5$。由于在新的网络中，新添加的层可以学习到零映射，所以所有在$\mathcal{F}_3$中的函数都可以在$\mathcal{F}_5$中找到，这就意味着$\mathcal{F}_3$嵌套在$\mathcal{F}_5$中。
  - **提高模型的性能**：虽然更深的网络可以通过恒等映射达到与浅层网络相同的性能，但只要新添加的层不完全学习到零映射，它们就有可能找到更好的解，从而提高模型的性能。
  - 假设我们的任务是识别图片中的猫。在一个浅层网络中，最后一层可能需要直接从边缘特征（如线段、色彩块等）识别出猫。但在一个深层网络中，中间的层可以先将边缘特征组合成部分特征（如眼睛、耳朵等），然后再由更高的层从部分特征识别出猫，这样可能会使识别更为准确。
--------------

## 7.6.2 残差块

让我们聚焦于神经网络局部：如图7.6.2 所示，假设我们的原始输入为$x$，而希望学出的理想映射为$f(\mathbf{x})$（作为 图7.6.2上方激活函数的输入）。
图7.6.2左图虚线框中的部分需要直接拟合出该映射$f(\mathbf{x})$，而右图虚线框中的部分则需要拟合出残差映射$f(\mathbf{x}) - \mathbf{x}$。
残差映射在现实中往往**更容易优化**。
以本节开头提到的恒等映射作为我们希望学出的理想映射$f(\mathbf{x})$，我们只需将图7.6.2 中右图虚线框内上方的加权运算（如仿射）的权重和偏置参数设成0，那么$f(\mathbf{x})$即为恒等映射。
实际中，**当理想映射$f(\mathbf{x})$极接近于恒等映射时，残差映射也易于捕捉恒等映射的细微波动**。
图7.6.2 右图是ResNet的基础架构--**残差块（residual block）**。
在残差块中，输入可通过跨层数据线路更快地向前传播。

<center>
    <img src="../img/residual-block.svg" alt="一个正常块（左图）和一个残差块（右图）">
</center>
<center>
    图7.6.2 一个正常块（左图）和一个残差块（右图）
</center>
ResNet沿用了VGG完整的$3\times 3$卷积层设计。
残差块里首先有2个有相同输出通道数的$3\times 3$卷积层。
每个卷积层后接一个批量规范化层和ReLU激活函数。
然后我们通过<b>跨层数据通路</b>，跳过这2个卷积运算，将输入直接加在最后的ReLU激活函数前。
这样的设计<b>要求2个卷积层的输出与输入形状一样，从而使它们可以相加</b>。
如果想改变通道数，就需要引入一个额外的$1\times 1$卷积层来将输入变换成需要的形状后再做相加运算。

- **要点：**
  - **残差块：** 残差块是ResNet基础架构的核心组件。在这个架构中，输入可以通过跨层数据线路更快地向前传播。
  - **理想映射与残差映射：** 在一个残差块内，我们希望学习的理想映射为$f(\mathbf{x})$。左侧部分需要直接拟合出该映射，而右侧部分则需要拟合出残差映射$f(\mathbf{x}) - \mathbf{x}$。
  - **优化：** 残差映射通常更容易优化。当理想映射$f(\mathbf{x})$接近恒等映射时，残差映射更易于捕捉到恒等映射的细微波动。
  - **设计：** ResNet采用了VGG完整的$3\times 3$卷积层设计。在残差块中，首先有两个相同输出通道数的$3\times 3$卷积层，每个卷积层后接一个批量规范化层和ReLU激活函数。然后，通过跨层数据通路，将输入直接加在最后的ReLU激活函数之前。
  - **形状一致性：** 这种设计需要两个卷积层的输出与输入形状一样，从而使它们可以相加。如果需要改变通道数，则需要引入一个额外的$1\times 1$卷积层来将输入转换成所需的形状后再进行相加运算。

- 残差块的实现如下：

In [2]:
%matplotlib inline
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l


class Residual(nn.Module):  #@save
    def __init__(self, input_channels, num_channels,
                 use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels,
                               kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels,
                               kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels,
                                   kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)

如图7.6.3所示，此代码生成两种类型的网络：
- 一种是当`use_1x1conv=False`时，应用ReLU非线性函数之前，将输入添加到输出。
- 另一种是当`use_1x1conv=True`时，添加通过$1 \times 1$卷积调整通道和分辨率。

<center>
    <img src="../img/resnet-block.svg" alt="一个单隐藏层的多层感知机，具有5个隐藏单元">
</center>
<center>
   图7.6.3 包含以及不包含 $1 \times 1$ 卷积层的残差块
</center>


- 下面查看输入和输出形状一致的情况：


In [3]:
blk = Residual(3,3)
X = torch.rand(4, 3, 6, 6)
Y = blk(X)
Y.shape

torch.Size([4, 3, 6, 6])

- 也可以在**增加输出通道数的同时，减半输出的高和宽**。


In [4]:
'''
此处虽然卷积层的输出与原始数据X的形状不一致了，就无法再与X相加。注意上图！
但是通过1X1卷积层将X的高宽减半，这样仍是一致的。
因此灵活之处在于可以调整X的高宽。
'''
blk = Residual(3,6, use_1x1conv=True, strides=2)
blk(X).shape

torch.Size([4, 6, 3, 3])

## 7.6.3 ResNet模型

- ResNet的前两层跟之前介绍的GoogLeNet中的一样：
  - 在输出通道数为64、步幅为2的$7 \times 7$卷积层后，接步幅为2的$3 \times 3$的最大池化层。
  - 不同之处在于ResNet每个卷积层后增加了批量规范化层。


In [5]:
## 注意Conv2d第一个参数为1，表示只是针对单通道图片，即灰度图片
## 本例中，b1输入的特征数据大小为224，输出变为56
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
                   nn.BatchNorm2d(64), nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

- GoogLeNet在后面接了4个由Inception块组成的模块。
- ResNet则使用4个由残差块组成的模块，每个模块使用若干个同样输出通道数的残差块。
  - 第一个模块的通道数同输入通道数一致。
  - 由于之前已经使用了步幅为2的最大池化层，所以无须减小高和宽。
  - 之后的每个模块在第一个残差块里将上一个模块的通道数翻倍，并将高和宽减半。

- 下面实现这个模块。
  - 注意，此处对第一个模块做了特别处理。


In [7]:
def resnet_block(input_channels, num_channels, num_residuals,
                 first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(input_channels, num_channels,
                                use_1x1conv=True, strides=2))
        else:
            blk.append(Residual(num_channels, num_channels))
    return blk

- 接着在ResNet加入所有残差块，这里每个模块使用2个残差块。


In [8]:
b2 = nn.Sequential(*resnet_block(64, 64, 2, first_block=True))
b3 = nn.Sequential(*resnet_block(64, 128, 2))
b4 = nn.Sequential(*resnet_block(128, 256, 2))
b5 = nn.Sequential(*resnet_block(256, 512, 2))

- 最后，与GoogLeNet一样，在ResNet中加入全局平均池化层，以及全连接层输出。


In [9]:
net = nn.Sequential(b1, b2, b3, b4, b5,
                    nn.AdaptiveAvgPool2d((1,1)),
                    nn.Flatten(), nn.Linear(512, 10))

- 每个模块有4个卷积层（不包括恒等映射的$1\times 1$卷积层）。
加上第一个$7\times 7$卷积层和最后一个全连接层，共有18层。
因此，这种模型通常被称为**ResNet-18**。

- 通过配置不同的通道数和模块里的残差块数可以得到不同的ResNet模型，例如更深的含152层的**ResNet-152**。
- 虽然ResNet的主体架构跟GoogLeNet类似，但**ResNet架构更简单，修改也更方便**。
- 这些因素都导致了ResNet迅速被广泛使用。
- 图7.6.4描述了完整的ResNet-18。


<center>
    <img src="../img/resnet18.svg" alt="ResNet-18 架构">
</center>
<center>
    图7.6.4 ResNet-18 架构
</center>

- 在训练ResNet之前，先观察一下ResNet中不同模块的输入形状是如何变化的。
- 在之前所有架构中，分辨率降低，通道数量增加，直到全局平均池化层聚集所有特征。

In [10]:
##如果第二个参数是3，则表示彩色图片，此处是灰度图片进行训练
##第一个参数表示图片个数，此处只有1张
X = torch.rand(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 128, 28, 28])
Sequential output shape:	 torch.Size([1, 256, 14, 14])
Sequential output shape:	 torch.Size([1, 512, 7, 7])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 512, 1, 1])
Flatten output shape:	 torch.Size([1, 512])
Linear output shape:	 torch.Size([1, 10])


## 7.6.4 训练模型

- 在Fashion-MNIST数据集上训练ResNet。


In [13]:
import time
import numpy as np
lr, num_epochs, batch_size = 0.05, 10, 256
start = time.time()
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())
end = time.time()
print(f'3090ti上模型训练耗时：{int(np.floor((end-start)/60))}分钟{(int(end-start)%60) }秒')

<img src='..\img\7_6_1.png' height=350 width=350>

- ResNet在训练速度、训练与测试精度方面都有很大提高

## 小结

* 学习嵌套函数（nested function）是训练神经网络的理想情况。在深层神经网络中，学习另一层作为恒等映射（identity function）较容易（尽管这是一个极端情况）。
* 残差映射可以更容易地学习同一函数，例如将权重层中的参数近似为零。
* 利用残差块（residual blocks）可以训练出一个有效的深层神经网络：输入可以通过层间的残余连接更快地向前传播。
* 残差网络（ResNet）对随后的深层神经网络设计产生了深远影响。